In [121]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob
import pandas as pd

PATH = ""

In [123]:
PATH = ""
!ls {PATH}

fastai
labeled_text
models
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp
train.tsv


In [128]:
df_train = pd.read_csv(f'{PATH}train.tsv', sep="\t")

In [129]:
df_test = pd.read_csv(f'{PATH}test.tsv',sep="\t")

In [130]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [131]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


# Language Model

In [132]:
#drop incomplete sentences in training set
def dropIncomplete (df, label="SentenceId", keepLocation=0):
    sentences = []
    for i in df[label].unique():  
        sentences.append(df[df[label] == i].reset_index(drop=True).loc[keepLocation])
    df_result = pd.DataFrame(sentences)
    return df_result

In [133]:
df_train = dropIncomplete(df_train)

In [134]:
df_test = dropIncomplete(df_test)

In [135]:
df_train = pd.DataFrame(df_train["Phrase"]).reset_index(drop=True)
df_test = pd.DataFrame(df_test["Phrase"]).reset_index(drop=True)

In [136]:
df_train.head()

,Phrase
0,A series of escapades demonstrating the adage ...
1,"This quiet , introspective and entertaining in..."
2,"Even fans of Ismail Merchant 's work , I suspe..."
3,A positively thrilling combination of ethnogra...
4,Aggressive self-glorification and a manipulati...


In [137]:
df_test.head()

,Phrase
0,An intermittently pleasing but mostly routine ...
1,Kidman is really the only thing that 's worth ...
2,Once you get into its rhythm ... the movie bec...
3,I kept wishing I was watching a documentary ab...
4,"Kinnear does n't aim for our sympathy , but ra..."


In [138]:
df_full = df_train.append(df_test).reset_index(drop=True)

In [139]:
df_full[df_full.shape[1]-10:]

,Phrase
11830,"At its worst , it implodes in a series of very..."
11831,Maybe I found the proceedings a little bit too...
11832,Moretti 's compelling anatomy of grief and the...
11833,Montias ... pumps a lot of energy into his nic...
11834,Not sweet enough to liven up its predictable s...
11835,"Nasty , ugly , pointless and depressing , even..."
11836,"With tightly organized efficiency , numerous f..."
11837,They should have called it Gutterball .
11838,"A long-winded , predictable scenario ."


In [140]:
df_full.shape

(11839, 1)

In [141]:
spacy_tok = spacy.load('en')

In [142]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [143]:
bs=64; bptt=70

In [144]:
df_train.shape, df_test.shape

((8529, 1), (3310, 1))

In [145]:
md = LanguageModelData.from_dataframes(path=PATH,field=TEXT,col="Phrase",train_df=df_train, val_df=df_test,
                                       test_df=df_test,bs=bs, bptt=bptt)

In [146]:
pickle.dump(TEXT, open(f'{PATH}TEXT.pkl','wb'))

In [147]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp/
train.tsv


In [148]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(42, 15364, 1, 193010)

In [149]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '<', '>', 'eos', '.', 'the', ',', 'a', 'and', 'of', 'to']

In [150]:
TEXT.vocab.stoi['to']

11

In [151]:
em_sz = 200
nh = 500
nl = 3

In [152]:
opt_fn = partial(optim.Adam, betas = (0.7, 0.99))

In [153]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

In [154]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [155]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      6.954403   6.299073  
    1      6.281553   5.876824                            
    2      5.920499   5.804518                            
    3      5.776692   5.743025                            
    4      5.606106   5.600349                            
    5      5.453188   5.556259                            
    6      5.349419   5.533326                            
    7      5.306063   5.470198                            
    8      5.203112   5.369964                            
    9      5.084493   5.375473                            
    10     4.960781   5.319035                            
    11     4.849896   5.346584                            
    12     4.762601   5.342232                            
    13     4.703944   5.341238                            
    14     4.658045   5.33422                             



[array([5.33422])]

In [156]:
learner.save_encoder('adam1_movieReview_enc')

In [157]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss                            
    0      4.678274   5.305937  
    1      4.665965   5.211305                            
    2      4.596527   5.224018                            
    3      4.497864   5.255579                            
    4      4.387083   5.296661                            
    5      4.299023   5.330986                            
    6      4.212508   5.342638                            
    7      4.142816   5.358443                            
    8      4.092514   5.374074                            
    9      4.052414   5.36959                             



[array([5.36959])]

In [158]:
learner.save_encoder('adam3_movieReview_10_enc')

In [159]:
learner.load_encoder('adam3_movieReview_10_enc')

# Test

In [160]:
#use our torchtext field to numericalize it so we can feed it to our language model.
m = learner.model
ss = """This is not something that I will have"""
s = [TEXT.preprocess(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

'this is not something that i will have'

In [161]:
#steps to test a language model
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [162]:
#Top 10 predictions
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['been', 'a', 'seen', 'made', '.', 'enough', 'as', 'done', 'not', 'to']

In [163]:
#generate a bit more text
print(ss,"\n")
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res, *_ =m(n[0].unsqueeze(0))
print("...")

This is not something that I will have 

been a good movie . < eos > the film is a lot of fun , but it 's not ...


# Sentiment

In [164]:
#Process training set data with sentiment label
df_trainWLabels = pd.read_csv("train.tsv", sep="\t")
df_trainWLabels = dropIncomplete(df_trainWLabels)
df_trainWLabels.drop(["PhraseId", "SentenceId"], axis = 1,inplace=True)
df_trainWLabels.reset_index(inplace=True, drop=True)
df_trainWLabels.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,"This quiet , introspective and entertaining in...",4
2,"Even fans of Ismail Merchant 's work , I suspe...",1
3,A positively thrilling combination of ethnogra...,3
4,Aggressive self-glorification and a manipulati...,1


In [165]:
len(df_trainWLabels)

8529

In [166]:
def IntoLabeledFolders(df, folderPath, validRatio=0.3, contentIndex=0, labelIndex=1):
    count = 1
    L = len(df)
    cutPoint = int(round((1-validRatio)*L))

    for index, row in df.iterrows():
        phrase=row[contentIndex]
        label = row[labelIndex]
        if (count<cutPoint):
            filename = os.path.join(folderPath,"train", str(label), str(count))+".txt"
        else:
            filename = os.path.join(folderPath,"valid", str(label), str(count))+".txt"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename,"w") as f:
            f.write(phrase)
        count = count + 1

In [167]:
IntoLabeledFolders(df=df_trainWLabels, folderPath = PATH)

In [169]:
#Show labels ###Start here if already created labeled folders
labelList = !ls {PATH}train
labelList

['0', '1', '2', '3', '4']

In [172]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[755, 1550, 1153, 1620, 891]

In [173]:
#Review sentences
label = 3
fileIndex = 40
FileList = !ls {PATH}train/{label}
sample = !cat {PATH}train/{label}/{FileList[fileIndex]}
sample

['It throws quirky characters , odd situations , and off-kilter dialogue at us , all as if to say , `` Look at this !']

In [174]:
class MovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)
    
        

In [176]:
MovieReviewLabel = data.Field(sequential=False)

In [177]:
TEXT = pickle.load(open(f"{PATH}TEXT.pkl","rb")) #open in read and bineary mode

In [181]:
PATH

''

In [182]:
splits = MovieReview.splits(TEXT, MovieReviewLabel,PATH,train="train", test="valid")

In [183]:
t = splits[0].examples[5]

In [184]:
t.label, " ".join(t.text[:30])

('0',
 "godard 's ode to tackling life 's wonderment is a rambling and incoherent manifesto about the vagueness of topical excess ... in praise of love remains a ponderous and pretentious")

In [185]:
bs = 64; bptt = 70

In [186]:
md2 = TextData.from_splits(PATH, splits, bs)

In [187]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [188]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [189]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [190]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_10_enc')

In [191]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [192]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                 
    0      1.570381   1.560513   0.281979  
    1      1.527872   1.533104   0.320312                 
    2      1.52383    1.516022   0.331177                 
    3      1.485386   1.516234   0.326762                 
    4      1.495567   1.514158   0.324708                 
    5      1.460893   1.496546   0.336398                 
    6      1.46912    1.495505   0.340026                 
    7      1.433329   1.488321   0.338375                 
    8      1.440055   1.48782    0.339086                 
    9      1.415417   1.480776   0.342598                 
    10     1.41844    1.479311   0.340468                 
    11     1.389164   1.47162    0.346149                 
    12     1.397917   1.485084   0.348587                 
    13     1.350888   1.474763   0.353885                 



[array([1.47476]), 0.35388513513513514]